In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/shahzadtalha843/Data_Science_Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="shahzadtalha843"
os.environ["MLFLOW_TRACKING_PASSWORD"]="c421606651d58b3b3979bedfa524d99f61a8214f"

In [2]:
import os
%pwd

'd:\\Mlops_course\\15-End_To_End_Data_Science_Project\\Data_Science_Project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Mlops_course\\15-End_To_End_Data_Science_Project\\Data_Science_Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/shahzadtalha843/Data_Science_Project.mlflow"


        )
        return model_evaluation_config


In [7]:
import os
import pandas as pd
from sklearn.metrics import  mean_absolute_error,mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Just log the model (no registry name)
            mlflow.sklearn.log_model(model, "model")

    


In [12]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation_config=ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-10-21 00:59:15,384: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-21 00:59:15,390: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-21 00:59:15,397: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-21 00:59:15,401: INFO: common: created directory at: artifacts]
[2025-10-21 00:59:15,405: INFO: common: created directory at: artifacts/model_evaluation]
[2025-10-21 00:59:18,090: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\HP\miniconda3\envs\mlflow\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [9]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: https://dagshub.com/shahzadtalha843/Data_Science_Project.mlflow


In [10]:
import mlflow
print("mlflow version:", mlflow.__version__)

mlflow version: 2.2.2


In [11]:
import mlflow, os
print("URI:", mlflow.get_tracking_uri())
print("USER:", os.getenv("MLFLOW_TRACKING_USERNAME"))


URI: https://dagshub.com/shahzadtalha843/Data_Science_Project.mlflow
USER: shahzadtalha843
